<a href="https://colab.research.google.com/github/jared-martin/higher-order-python/blob/main/Higher_Order_Python_Chapter_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Recursion and Callbacks

Recursion is a method of solving a problem by reducing it to a simpler problem of the same type.

### 1.1 Decimal To Binary Conversion

Any whole number has the form `2k + b`, where `k` is some smaller whole number and `b` is either 0 or 1. `b` is the final bit of the binary expansion. It’s easy to see whether this final bit is 0 or 1; just look to see whether the input number is even or odd. The rest of the number is `2k`, whose binary expansion is the same as that of `k`, but shifted left one place.

In [1]:
def binary(n: int) -> str:
    if n in (0, 1):
        return str(n)
    else:
        k, b = divmod(n, 2)
        return binary(k) + str(b)

binary(37)

'100101'

Note that the above will only work for a non-negative integer.  We could add type checking at the start of the function call, but then it would check the type of all the values of `k` on all the recursive calls.  But if `n` is a non-negative integer then we know `k` is a non-negative integer, so we don't *need* to check type recursively.

One solution would be to make the above function a private method, and create a *different* public method that checks the type of `n` before calling the above function.

As a mental exercise, I implemented binary expansion using a generator:

In [3]:
from typing import Generator


def gen_binary_bits(n: int) -> Generator[str, None, None]:
    if n in (0, 1):
        yield str(n)
    else:
        k, b = divmod(n, 2)
        yield from binary(k)
        yield str(b)

''.join(gen_binary_bits(37))

'100101'

### 1.2 Factorial

Suppose you have a list of `n` different items. For concreteness, we’ll suppose that these items are letters of the alphabet. How many different orders are there for such a list? Obviously, the answer depends on `n`, so it is a function of `n`. This function is called the factorial function. The factorial of `n` is the number of different orders for a list of `n` different items. Mathematicians usually write it as a postfix (!) mark, so that the factorial of `n` is `n!`. They also call the different orders permutations.

In [19]:
def factorial(n: int) -> int:
    if n == 0:
        return 1
    else:
        return factorial(n - 1) * n

factorial(4)

24

I also implemented this using a generator, as a mental exercise:

In [20]:
def gen_factorial_factors(n: int) -> Generator[int, None, None]:
    if n == 0:
        yield 1
    else:
        # If you put this after the recursive call, it reverses the order!
        yield n
        yield from gen_factorial_factors(n - 1)

list(gen_factorial_factors(4))

[4, 3, 2, 1, 1]

Doing it this way made me notice something.  If we make the base case `n == 0`, it does the last operation (multiplying by 1) one extra time.  We can fix this by making the base case `n <= 1`.

In [21]:
def gen_factorial_factors(n: int) -> Generator[int, None, None]:
    if n <= 1:  # Improved base case
        yield 1
    else:
        # If you put this after the recursive call, it reverses the order!
        yield n
        yield from gen_factorial_factors(n - 1)

list(gen_factorial_factors(4))

[4, 3, 2, 1]

We can then find the factorial by reducing the generator using multiplication:

In [22]:
import functools
import operator


def silly_factorial(n: int) -> int:
    return functools.reduce(operator.mul, gen_factorial_factors(n))

silly_factorial(4)

24

Since the book motivates the discussion of factorial by talking about permutations, I decided to try implementing a permutation generator too:

In [30]:
def gen_permutations(items: list) -> Generator[list, None, None]:
    if len(items) <= 1:
        yield items
    elif len(items) == 2:
        yield items
        yield list(reversed(items))
    else:
        item, *other_items = items
        for permutation in gen_permutations(other_items):
            # Insert one item at each spot in each permutation of the others
            for index in range(len(permutation) + 1):
                yield permutation[:index] + [item] + permutation[index:]

items = list('ABCD')
permutations = list(map(''.join, gen_permutations(items)))

assert(len(permutations) == factorial(len(items)))
permutations

['ABCD',
 'BACD',
 'BCAD',
 'BCDA',
 'ACBD',
 'CABD',
 'CBAD',
 'CBDA',
 'ACDB',
 'CADB',
 'CDAB',
 'CDBA',
 'ABDC',
 'BADC',
 'BDAC',
 'BDCA',
 'ADBC',
 'DABC',
 'DBAC',
 'DBCA',
 'ADCB',
 'DACB',
 'DCAB',
 'DCBA']

### 1.3 The Tower of Hanoi

The puzzle has three pegs, called A, B, and C. On peg A is a tower of disks of graduated sizes, with the largest on the bottom and the smallest on the top. The puzzle is to move the entire tower from A to C, subject to the following restrictions: you may move only one disk at a time, and no disk may ever rest atop a smaller disk. The number of disks varies depending on who is posing the problem, but it is traditionally 64. We will try to solve the problem in the general case, for `n` disks.

In [34]:
def hanoi(n: int, start, end, extra):
    if (n == 1):
        # If the "tower" is actually only one disk high, just move it
        print(f'Move disk #1 from {start} to {end}')
    else:
        # Move all the disks except for disk n (the Big Disk) from the start
        # peg to the extra peg, using this method
        hanoi(n - 1, start, extra, end)
        # Move disk n (the Big Disk) from the start peg to the end peg
        print(f'Move disk #{n} from {start} to {end}')
        # Move all the other disks from the extra peg to the end peg, using this
        # method
        hanoi(n - 1, extra, end, start)

hanoi(3, *list('ABC'))

Move disk #1 from A to B
Move disk #2 from A to C
Move disk #1 from B to C
Move disk #3 from A to B
Move disk #1 from C to A
Move disk #2 from C to B
Move disk #1 from A to B


If we wanted a graphic display of moving disks instead of a simple printout of instructions, we could replace the print statements with something fancier. But we can make the software more flexible almost for free by parametrizing the output behavior. Instead of having a print statement hardwired in, `hanoi()` will accept an extra argument that is a function that will be called each time `hanoi()` wants to move a disk. This function will print an instruction, or update a graphical display, or do whatever else we want. The function will be passed the number of the disk, and the source and destination pegs.

In [64]:
from collections.abc import Callable


def hanoi_with_callback(n: int, start, end, extra, move_disk: Callable):
    if (n == 1):
        move_disk(1, start, end)
    else:
        hanoi_with_callback(n - 1, start, extra, end, move_disk)
        move_disk(n, start, end)
        hanoi_with_callback(n - 1, extra, end, start, move_disk)


def print_instruction(disk: int, start, end):
    print(f'Move disk #{disk} from {start} to {end}')

hanoi_with_callback(3, *list('ABC'), print_instruction)

Move disk #1 from A to B
Move disk #2 from A to C
Move disk #1 from B to C
Move disk #3 from A to B
Move disk #1 from C to A
Move disk #2 from C to B
Move disk #1 from A to B


### 1.4 Hierarchical Data
Most recursive functions are built to deal with recursive data structures. A recursive data structure is one like a list, tree, or heap that is defined in terms of simpler instances of the same data structure. The most familiar example is probably a file system directory structure.

In [39]:
from pathlib import Path


def total_size(top: Path) -> int:
    total = top.stat().st_size
    if top.is_file():
        return total
    else:
        for path in top.iterdir():
            total += total_size(path)
    return total

total_size(Path('.'))

56889676

### 1.5 Applications and Variations of Directory Walking

We had written a function, `total_size()`, which contained useful functionality: it walked a directory tree recursively. If we could cleanly separate the directory-walking part of the code from the total-size-computing part, then we might be able to re-use the directory-walking part in many other projects for many other purposes. How can we separate the two functionalities?

In [41]:
def dir_walk(top: Path, code: Callable):
    code(top)
    if top.is_dir():
        for path in top.iterdir():
            dir_walk(path, code)

dir_walk(Path('.'), print)

.
.config
.config/.last_update_check.json
.config/.last_opt_in_prompt.yaml
.config/active_config
.config/.last_survey_prompt.yaml
.config/config_sentinel
.config/configurations
.config/configurations/config_default
.config/logs
.config/logs/2022.02.01
.config/logs/2022.02.01/14.30.57.022317.log
.config/logs/2022.02.01/14.31.40.709264.log
.config/logs/2022.02.01/14.31.33.364834.log
.config/logs/2022.02.01/14.31.58.218326.log
.config/logs/2022.02.01/14.31.16.993813.log
.config/logs/2022.02.01/14.31.57.576848.log
.config/gce
sample_data
sample_data/anscombe.json
sample_data/README.md
sample_data/california_housing_train.csv
sample_data/mnist_train_small.csv
sample_data/mnist_test.csv
sample_data/california_housing_test.csv


### 1.6 Functional Versus Object-Oriented Programming

Now let’s back up a moment and look at what we did. We had a useful function, total_size(), which contained code for walking a directory structure that was going to be useful in other applications. So we made total_size() more general by pulling out all the parts that related to the computation of sizes, and replacing them with calls to arbitrary user-specified functions. The result was dir_walk(). Now, for any program that needs to walk a directory structure and do something, dir_walk() handles the walking part, and the argument functions handle the “do something” part. By passing the appropriate pair of functions to dir_walk(), we can make it do whatever we want it to. We’ve gained flexibility and the chance to re-use the dir_walk() code by factoring out the useful part and parametrizing it with two functional arguments. This is the heart of the functional style of programming

Since there's no code in this section, I'll use it as an opportunity to implement `dir_walk()` with a generator so that `code()` can be mapped over it instead of passed to it as a callback.

In [49]:
def gen_paths_depth_first(top: Path) -> Generator[Path, None, None]:
    yield top
    if top.is_dir():
        for path in top.iterdir():
            yield from gen_paths_depth_first(path)

for path in gen_paths_depth_first(Path('.')):
    print(path)


# You could even do this breadth first
from collections import deque


def gen_paths_breadth_first(top: Path) -> Generator[Path, None, None]:
    paths = deque([top])
    while len(paths) > 0:
        # FIFO - if you pop it off the right instead, it's a DFS!
        path = paths.popleft()
        yield path
        if path.is_dir():
            for subpath in path.iterdir():
                paths.append(subpath)

.
.config
.config/.last_update_check.json
.config/.last_opt_in_prompt.yaml
.config/active_config
.config/.last_survey_prompt.yaml
.config/config_sentinel
.config/configurations
.config/configurations/config_default
.config/logs
.config/logs/2022.02.01
.config/logs/2022.02.01/14.30.57.022317.log
.config/logs/2022.02.01/14.31.40.709264.log
.config/logs/2022.02.01/14.31.33.364834.log
.config/logs/2022.02.01/14.31.58.218326.log
.config/logs/2022.02.01/14.31.16.993813.log
.config/logs/2022.02.01/14.31.57.576848.log
.config/gce
sample_data
sample_data/anscombe.json
sample_data/README.md
sample_data/california_housing_train.csv
sample_data/mnist_train_small.csv
sample_data/mnist_test.csv
sample_data/california_housing_test.csv


### 1.7 HTML

`TODO`

### 1.8 When Recursion Blows Up

Sometimes a problem appears to be naturally recursive, and then the recursive solution is grossly inefficient. A very simple example arises when you want to compute Fibonacci numbers. This is a rather unrealistic example, but it has the benefit of being very simple. We’ll see a more practical example of the same thing in Section 3.7.

#### 1.8.2 Fibonacci Numbers

Fibonacci numbers are named for Leonardo of Pisa, whose nickname was Fibonacci, who discussed them in the 13th century in connection with a mathematical problem about rabbits. Initially, you have one pair of baby rabbits. Baby rabbits grow to adults in one month, and the following month they produce a new pair of baby rabbits, making two pairs.

Assuming no rabbits die, and rabbit production continues, how many pairs of rabbits are there in each month?

In [57]:
def fib(month: int) -> int:
    # XXX: This is a bad implementation!
    if month < 2:
        return 1
    else:
        return fib(month - 1) + fib(month - 2)

print(fib(35))  # Takes 4 seconds!

14930352


This is perfectly straightforward, but it has a problem: except for small arguments, it takes forever. If you ask for `fib(25)`, for example, it needs to make recursive calls to compute `fib(24)` and `fib(23)`. But the call to `fib(24)` also makes a recursive call to `fib(23)`, as well as another to compute `fib(22)`. Both calls to `fib(23)` will also call `fib(22)`, for a total of three times. It turns out that `fib(21)` is computed 5 times, `fib(20)` is computed 8 times, and `fib(19)` is computed 13 times.

#### 1.8.2 Partitioning

Here’s a somewhat more realistic example. We have some valuable items, which we’ll call “treasures,” and we want to divide them evenly between two people. We know the value of each item, and we would like to ensure that both people get collections of items whose total value is the same. Or, to recast the problem in a more mundane light: we know the weight of each of the various groceries you bought today, and since you’re going to carry them home with one bag in each hand, you want to distribute the weight evenly.

This problem is called the partition problem. We’ll generalize the problem a little: instead of trying to divide a list of treasures into two equal parts, we’ll try to find some share of the treasures whose total value is a given target amount. Finding an even division of the treasures is the same as finding a share whose value is half of the total value of all the treasures; then the other share is the rest of the treasures, whose total value is the same.

In [67]:
def find_share(target: int, treasures: tuple) -> tuple:
    # XXX: This is a bad implementation!
    if target == 0:
        return ()
    elif target < 0 or len(treasures) == 0:
        return None
    else:
        first, *rest = treasures
        solution = find_share(target - first, rest)
        if solution is not None:
            return (first, *solution)
        else:
            return find_share(target, rest)

find_share(11, (1, 2, 3, 8))

(1, 2, 8)

The idea of ignoring the first treasure and looking for a solution among the remaining treasures, thus reducing the problem to a simpler case, is natural. A solution without recursion would probably end up duplicating the underlying machinery of the recursive solution, and simulating the behavior of the function- call stack manually.

Now solving the partition problem is easy; it’s a call to `find_share()`, which finds the first share, and then some extra work to compute the elements of the original array that are not included in the first share:

In [69]:
from collections import Counter


def relative_complement_preserving_dupes(tup1: tuple, tup2: tuple) -> tuple:
    """Return elements in the first tuple that are not in the second.  Example:
    >>> relative_complement_preserving_dupes((1, 1, 2), (2, 3))
    (1, 1)
    """
    return tuple((Counter(tup1) - Counter(tup2)).elements())


def partition(treasures):
    total = sum(treasures)
    share_1 = find_share(total/2, treasures)
    if share_1 is None:
        return None
    else:
        share_2 = relative_complement_preserving_dupes(treasures, share_1)
        return (share_1, share_2)

partition((1, 2, 3, 4, 6))

((1, 3, 4), (2, 6))

The find_share function, however, has a problem: it takes much too long to run, especially if there is no solution. It has essentially the same problem as fib did: it repeats the same work over and over. For example, suppose it is trying to find a division of `1 2 3 4 5 6 7` with target sum 14. It might be investigating shares shares that contain 1 and 3, and then look to see if it can make `5 6 7` hit the target sum of 10. It can’t, so it will look for other solutions. Later on, it might investigate shares that contain 4, and again look to see if it can make `5 6 7` hit the target sum of 10. This is a waste of time; find_share should remember that `5 6 7` cannot hit a target sum of 10 from the first time it investigated that.